In [1]:
def inverse_standardize(data, means, stds):
    """
    对标准化数据进行反标准化。

    参数：
    data: np.ndarray - 标准化后的数据。
    means: np.ndarray - 每个通道的均值。
    stds: np.ndarray - 每个通道的标准差。

    返回：
    original_data: np.ndarray - 反标准化后的数据。
    """
    original_data = data * stds + means
    return original_data


In [2]:
import numpy as np

mean = np.load('../data/dt1/mean.npy')
std = np.load('../data/dt1/std.npy')
predictions_data_single=np.load('./data/single/predictions.npy')
predictions_inverse = np.zeros_like(predictions_data_single)  # 创建一个与 pre_data_1 形状相同的数组
for i in range(predictions_data_single.shape[1]):
    predictions_inverse[:, i] = inverse_standardize(predictions_data_single[:, i], mean[i], std[i])
# 保存反标准化后的数据
np.save('data/single/predictions_inverse.npy', predictions_inverse)
print("反标准化后的数据已保存为 'data/predictions_inverse.npy'")

反标准化后的数据已保存为 'data/predictions_inverse.npy'


In [3]:
predictions_inverse.shape

(600, 8)

In [4]:
from Ablation.model import AnticipationModule
def anticipate_mode_ori(data_path):
    predication = np.load(data_path)
    anticipation_module = AnticipationModule()
    prediction_data = anticipation_module(predication)
    return prediction_data
anticipate_data = anticipate_mode_ori('data/single/predictions_inverse.npy')
print(anticipate_data.shape)
np.save('data/single/predictions_inverse_final.npy', anticipate_data)

(600, 1)


In [7]:
import pandas as pd
def extract_label():
    file_path = 'E:/Dataset/waves/'
    file_path_ori = file_path + 'dataset1.csv'
    data = pd.read_csv(file_path_ori)
    # 忽略 'Date/Time' 列
    if 'Date/Time' in data.columns:
        data = data.drop(columns=['Date/Time'])
    mwh_data = data['MWH'].values[-600:]  # 提取最后 600 行的 MWH 列
    mwh_data=mwh_data.reshape(-1, 1) 
    return mwh_data

In [8]:
original_data = extract_label()
original_data.shape

(600, 1)

In [9]:
from Ablation.model import ModelEvaluator
#源数据的数据评测
predictions_inverse_final=np.load('data/single/predictions_inverse_final.npy')
evaluator = ModelEvaluator(predictions_inverse_final, original_data)
print(f"Root Mean Squared Error (RMSE): {evaluator.rmse()}")
print(f"Mean Absolute Error (MAE): {evaluator.mae()}")
print(f"Sum of Squared Errors (SSE): {evaluator.sse()}")
print(f"Mean Absolute Percentage Error (MAPE): {evaluator.mape()}")
print(f"Theil's Inequality Coefficient (TIC): {evaluator.tic()}")

Root Mean Squared Error (RMSE): 0.14983896174524994
Mean Absolute Error (MAE): 0.11568351312239965
Sum of Squared Errors (SSE): 13.471028674136686
Mean Absolute Percentage Error (MAPE): 4.830199890085517
Theil's Inequality Coefficient (TIC): 0.05624039376421019
